In [ ]:
# extracting movies into frames folder
# DOESN'T SEEM TO WORK
import os

for i in range(12):
    mov = i + 32
    command = "ffmpeg -loglevel error -i ./Desktop/movies2/MEDIAEVAL18_{}.mp4 -r 1 -f image2 ./Desktop/frames2/{}/frame-%05d.jpg".format(mov, mov)
    os.system(command)

In [63]:
# here is the code for generating the feature and label matrices!

# IMPORTING THE BACKGROUND STUFF
import cv2
import numpy as np
import os, math, csv
from keras.models import Model
from keras.applications.resnet50 import ResNet50

# A helper function for later down the road --> for converting fear annotations to one-hot vectors
def fear_oneHot(movie_length, fear_path):
    # load start and end times from file
    y_data = np.loadtxt(fear_path, skiprows=1)
    y_data_input = np.zeros((movie_length))
    
    # need to address corner case if y_data is 1D array
    if (y_data.shape[0] > 1) and (type(y_data[0]) == np.float64):
        y_data_ext = np.zeros([1,2])
        y_data_ext[:,:] = np.asarray(y_data)
    else:
        y_data_ext = y_data 
               
    # for each element in first dimension of the y_data array
    for i in range(y_data_ext.shape[0]):
        # access the start time number and end time number
        start = int(y_data_ext[i][0])
        end = int(y_data_ext[i][1])
        # set the elements between these indices in the zeros array to one
        y_data_input[start] = 1 #maybe superfluous
        y_data_input[end] = 1
        y_data_input[start:end] = 1
        
        
    return y_data_input

In [22]:
# FOR LOADING THE VGG16 DATA

VGG_data_dim = 4096
timesteps= 6262 # the max number of timesteps --> note that the movies have different lengths
num_movies = 12 #TODO - THE NUMBER OF MOVIES IN THE FOLDER NEEDS TO BE CONFIRMED FOR EACH DEVSET
VGG16_feature_data = np.zeros([num_movies, timesteps, VGG_data_dim])

# for each movie
for num in range(num_movies):
    # load the X_input data
    # get the file path
    mov = 54 + num # TODO - MAY BE NECESSARY TO ADD A CONSTANT IN ORDER TO GET CORRECT MOVIE NUMBER
    # TODO - PLEASE CONFIRM FILE PATH
    if mov < 10:
        path = os.path.join('/Users/chloeloughridge/Desktop/AI Grant Project/Dataset/2018 Task Data/MEDIAEVAL18-TestSet-Visual_features/visual_features/MEDIAEVAL18_0{}/fc6'.format(mov))
    if mov > 10:
        path = os.path.join('/Users/chloeloughridge/Desktop/AI Grant Project/Dataset/2018 Task Data/MEDIAEVAL18-TestSet-Visual_features/visual_features/MEDIAEVAL18_{}/fc6'.format(mov))
    count = 0
    # loading the VGG16 feature data from each file in the folder
    for file in os.listdir(path):
        filename = os.fsdecode(file)
        if filename.endswith(".txt"):
            input_data = np.loadtxt(os.path.join(path, file), delimiter=',')
            VGG16_feature_data[num, count, :] = np.asarray(input_data)[:]
            #print(count)
            count = count + 1
            #continue
        #else:
            #continue
    print(mov)
            
print(count)
print(VGG16_feature_data.shape)
print(VGG16_feature_data)

54
55
56
57
58
59
60
61
62
63
64
65
5564
(12, 6262, 4096)
[[[ -4.4444    0.76968  -3.2783  ...   0.23307  -1.0833   -0.45209]
  [ -4.5598    0.79331  -3.3777  ...   0.30708  -1.0658   -0.51627]
  [ -6.067    -8.4672   -6.7512  ... -10.636    -9.7449    4.7505 ]
  ...
  [  0.        0.        0.      ...   0.        0.        0.     ]
  [  0.        0.        0.      ...   0.        0.        0.     ]
  [  0.        0.        0.      ...   0.        0.        0.     ]]

 [[ -4.8072    1.0747   -3.548   ...   0.33026  -1.3472   -0.25087]
  [ -4.9485    0.96946  -3.4589  ...   0.12901  -1.2983   -0.19144]
  [ -6.1164   -8.6679   -7.4226  ... -10.407   -10.222     4.948  ]
  ...
  [  0.        0.        0.      ...   0.        0.        0.     ]
  [  0.        0.        0.      ...   0.        0.        0.     ]
  [  0.        0.        0.      ...   0.        0.        0.     ]]

 [[ -4.7292    0.2542   -2.9289  ...   0.52958  -1.3883   -0.68409]
  [ -4.736     0.26421  -2.905   ...   0.4

In [23]:
# SAVING THE VGG16 FEATURE DATA

#file_name = None # TODO - Maybe something like VGG_feature_data01 for devset part 1?
#path = None # TODO - path for saving the file
#print(VGG16_feature_data)
np.save('../Desktop/VGG16_test.npy', VGG16_feature_data)

In [14]:
# splitting VGG data for DevSet part 2:
# we want two matrices of 15 movies each
VGG_1 = np.load('../Desktop/VGG16_data02.npy')[:15, :, :]
print(VGG_1.shape)
np.save('../Desktop/VGG16_data02_1.npy', VGG_1)
VGG_2 = np.load('../Desktop/VGG16_data02.npy')[15:, :, :]
np.save('../Desktop/VGG16_data02_2.npy', VGG_2)

(15, 6262, 4096)


In [18]:
# splitting labels into two matrices
# splitting VGG data for DevSet part 2:
# we want two matrices of 15 movies each
labels_1 = np.load('../Desktop/labels02.npy')[:15, :]
print(labels_1.shape)
np.save('../Desktop/labels02_1.npy', labels_1)
labels_2 = np.load('../Desktop/labels02.npy')[15:, :]
np.save('../Desktop/labels02_2.npy', labels_2)

(15, 6262)


In [20]:
# splitting audio into two matrices
# splitting VGG data for DevSet part 2:
# we want two matrices of 15 movies each
audio_1 = np.load('../Desktop/audio_data02.npy')[:15, :]
print(audio_1.shape)
np.save('../Desktop/audio_data02_1.npy', audio_1)
audio_2 = np.load('../Desktop/audio_data02.npy')[15:, :]
np.save('../Desktop/audio_data02_2.npy', audio_2)

(15, 6262, 1583)


In [22]:
np.save('../Desktop/VGG16_data01.npy', VGG16_feature_data)

In [64]:
# for something else entirely
predictions = np.load('../Desktop/prediction_test01')

In [65]:
for num in predictions[0]:
    print(num)

110.0
111.0
207.0
209.0
210.0
212.0
214.0
220.0
237.0
238.0
240.0
241.0
307.0
308.0
311.0
315.0
408.0
409.0
411.0
415.0
419.0
425.0
426.0
437.0
440.0
484.0
489.0
589.0
599.0
614.0
618.0
621.0
622.0
624.0
625.0
712.0
714.0
716.0
723.0
802.0
808.0
810.0
825.0
829.0
839.0
844.0
912.0
923.0
924.0
927.0
928.0
940.0
941.0
1015.0
1022.0
1025.0
1026.0
1116.0
1118.0
1120.0
1127.0
1129.0
1131.0
1145.0
1212.0
1215.0
1232.0
1234.0
1241.0
1246.0
1248.0
1306.0
1318.0
1320.0
1322.0
1324.0
1340.0
1429.0
1430.0
1432.0
1522.0
1524.0
1526.0
1527.0
1529.0
1531.0
1616.0
1623.0
1625.0
1629.0
1633.0
1641.0
1643.0
1645.0
1646.0
1717.0
1729.0
1818.0
1830.0
1919.0
1931.0
2020.0
2032.0
2121.0
2133.0
2222.0
2234.0
2323.0
2335.0
2424.0
2436.0
2525.0
2537.0
2626.0
2638.0
2727.0
2739.0
2828.0
2840.0
2929.0
2941.0
3030.0
3042.0
3131.0
3143.0
3232.0
3244.0
3333.0
3345.0
3434.0
3446.0
3535.0
3547.0
3636.0
3648.0
3737.0
3749.0
3838.0
3850.0
3939.0
3951.0
4040.0
4052.0
4141.0
4153.0
4242.0
4254.0
4343.0
4355.0
4444.0
445

In [66]:
record = np.zeros(predictions.shape)
for mov in range(12):
    mov_num = mov
    print('MEDIAEVAL18_{}'.format(mov_num + 54))
    testSet_lengths = [207, 207, 207, 1128, 451, 4653, 3256, 4622, 4729, 1664, 5465, 5559]
    #for i in range(predictions.shape[1]):
    i = 0
    while i < predictions.shape[1]:
        #print(predictions[mov_num][i])
        if predictions[mov_num][i] < testSet_lengths[mov_num] and predictions[mov_num][i] != 0:
            if predictions[mov_num][i+1] > testSet_lengths[mov_num]:
                print(predictions[mov_num][i], testSet_lengths[mov_num])
            else:
                print(predictions[mov_num][i], predictions[mov_num][i+1])
                i += 1
        i += 1

MEDIAEVAL18_54
110.0 111.0
MEDIAEVAL18_55
53.0 60.0
64.0 65.0
146.0 148.0
158.0 159.0
172.0 178.0
179.0 207
MEDIAEVAL18_56
56.0 58.0
69.0 71.0
98.0 207
MEDIAEVAL18_57
108.0 121.0
294.0 296.0
297.0 299.0
397.0 399.0
408.0 409.0
412.0 414.0
416.0 420.0
437.0 439.0
531.0 535.0
590.0 593.0
632.0 678.0
808.0 820.0
909.0 921.0
1010.0 1022.0
1111.0 1123.0
MEDIAEVAL18_58
85.0 93.0
95.0 111.0
195.0 196.0
206.0 210.0
215.0 312.0
314.0 404.0
405.0 407.0
408.0 410.0
416.0 419.0
MEDIAEVAL18_59
4.0 6.0
110.0 112.0
206.0 210.0
213.0 217.0
218.0 222.0
308.0 312.0
313.0 318.0
413.0 426.0
427.0 437.0
505.0 541.0
543.0 545.0
613.0 618.0
620.0 622.0
727.0 730.0
739.0 743.0
808.0 810.0
812.0 816.0
818.0 828.0
835.0 837.0
839.0 840.0
842.0 844.0
909.0 921.0
1010.0 1022.0
1111.0 1123.0
1212.0 1224.0
1313.0 1325.0
1414.0 1426.0
1515.0 1527.0
1616.0 1628.0
1717.0 1729.0
1818.0 1830.0
1919.0 1931.0
2020.0 2032.0
2121.0 2133.0
2222.0 2234.0
2323.0 2335.0
2424.0 2436.0
2525.0 2537.0
2626.0 2638.0
2727.0 2739.0
28

In [7]:
# FOR CREATING THE LABELS MATRIX

# please download Fear Annotations from Liris Accede dataset
num_movies = 30
timesteps = 6262
labels_matrix = np.zeros([num_movies, timesteps])

for mov in range(num_movies):
    num = mov + 14
    # create the appropriate path to the fear annotation data
    # TODO - PLEASE CONFIRM THESE ARE THE CORRECT PATHS
    if num < 10:
        path = os.path.join('/Users/chloeloughridge/Desktop/MEDIAEVAL18-DevSet-Part2-Fear-annotations/annotations/MEDIAEVAL18_0{}_Fear.txt'.format(num))
    if num > 10:
        path = os.path.join('/Users/chloeloughridge/Desktop/MEDIAEVAL18-DevSet-Part2-Fear-annotations/annotations/MEDIAEVAL18_{}_Fear.txt'.format(num))
    # create a one-hot vector
    y_data = fear_oneHot(timesteps, path)
    # add this one-hot vector to y_train_data
    labels_matrix[mov, :] = y_data
    print(num)

14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43


/Users/chloeloughridge/anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:13: UserWarning: loadtxt: Empty input file: "/Users/chloeloughridge/Desktop/MEDIAEVAL18-DevSet-Part2-Fear-annotations/annotations/MEDIAEVAL18_15_Fear.txt"
/Users/chloeloughridge/anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:13: UserWarning: loadtxt: Empty input file: "/Users/chloeloughridge/Desktop/MEDIAEVAL18-DevSet-Part2-Fear-annotations/annotations/MEDIAEVAL18_17_Fear.txt"
/Users/chloeloughridge/anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:13: UserWarning: loadtxt: Empty input file: "/Users/chloeloughridge/Desktop/MEDIAEVAL18-DevSet-Part2-Fear-annotations/annotations/MEDIAEVAL18_20_Fear.txt"
/Users/chloeloughridge/anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:13: UserWarning: loadtxt: Empty input file: "/Users/chloeloughridge/Desktop/MEDIAEVAL18-DevSet-Part2-Fear-annotations/annotations/MEDIAEVAL18_24_Fear.txt"
/Users/chloeloughridge/anaconda/lib/python3.5/site-p

In [10]:

np.save('../Desktop/labels02.npy', labels_matrix)

In [ ]:
# SAVING THE LABELS MATRIX

file_name = None # TODO - Maybe something like labels01 for devset part 1?
path = None # TODO - path for saving the file
np.save(file_name, path)

In [ ]:
# GENERATING RESNET50 FEATURE DATA

# The following code assumes that 1 frame per second from each movie has been saved to a folder on Desktop
# called "frames". Within "frames" I created a numbered folder for each movie.

# I ran the following terminal command for each movie to extract one frame per second
# the downside is that I then had to rerun this command for each movie in the dataset (and tweak the destination path
# accordingly). Perhaps there is a better way?

# Here is the command: 
# ffmpeg -loglevel error -i [MOVIE NAME].mp4 -r 1 -f [DESTINATION PATH] frame-%05d.jpg

In [4]:
# FOR GENERATING RESNET50 FEATURE DATA

# import pretrained InceptionV3 model from keras
model = ResNet50(include_top=False, weights='imagenet', input_tensor=None, input_shape=None, pooling=None, classes=1000)
print(model.summary())

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, None, None, 3 0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, None, None, 3 0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, None, None, 6 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_conv1 (BatchNormalization)   (None, None, None, 6 256         conv1[0][0]                      
__________________________________________________________________________________________________
activation

In [ ]:
# GENERATING RESNET50 FEATURE DATA CONTINUED
# iterate though each movie in the folder

timesteps = 6262

num_movies = 30 # TODO - however many movies are in this section of the dataset

# set up master array
resnet50_feature_data = np.zeros([num_movies, timesteps, 30720]) # TODO - this last number may be wrong; keras 
# will probably throw an enlightening error if this is the case

#for num in range(num_movies):
for num in range(num_movies):
    mov = num + 14
    count = 0
    for file in os.listdir('../Desktop/frames2/{}/'.format(mov)):
        filename = os.fsdecode(file)
        if filename.endswith(".jpg"):
            # extract the image data
            test_img = cv2.imread(os.path.join('../Desktop/frames2/{}/'.format(mov), filename))
            data = (test_img/255)[np.newaxis, :, :, :]
            # send image data through the resnet model
            intermediate_output = model.predict(data)
            # reshape model output
            feature_data = intermediate_output.flatten()
            # slot into the master away
            resnet50_feature_data[num, count, :feature_data.shape[0]] = feature_data
            print(count)
            count = count + 1
            continue
        else:
            continue
    print(num)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45


In [49]:
np.save('../Desktop/resnet0_6.npy', inception_v3_feature_data)

In [52]:
# merging the two parts of feature matrix
inception_v3_feature_data[7:, :212, :4096] = np.load('../Desktop/ResNet50_feature_data.npy')

In [57]:
np.save('../Desktop/resnet01.npy', inception_v3_feature_data)

In [55]:
for i in inception_v3_feature_data[0][0]:
    if i != 0:
        print("yay!")
    


print(inception_v3_feature_data[5][0][890])

yay!
yay!
yay!
yay!
yay!
yay!
yay!
yay!
yay!
yay!
yay!
yay!
yay!
yay!
yay!
yay!
yay!
yay!
yay!
yay!
yay!
yay!
yay!
yay!
yay!
yay!
yay!
yay!
yay!
yay!
yay!
yay!
yay!
yay!
yay!
yay!
yay!
yay!
yay!
yay!
yay!
yay!
yay!
yay!
yay!
yay!
yay!
yay!
yay!
yay!
yay!
yay!
yay!
yay!
yay!
yay!
yay!
yay!
yay!
yay!
yay!
yay!
yay!
yay!
yay!
yay!
yay!
yay!
yay!
yay!
yay!
yay!
yay!
yay!
yay!
yay!
yay!
yay!
yay!
yay!
yay!
yay!
yay!
yay!
yay!
yay!
yay!
yay!
yay!
yay!
yay!
yay!
yay!
yay!
yay!
yay!
yay!
yay!
yay!
yay!
yay!
yay!
yay!
yay!
yay!
yay!
yay!
yay!
yay!
yay!
yay!
yay!
yay!
yay!
yay!
yay!
yay!
yay!
yay!
yay!
yay!
yay!
yay!
yay!
yay!
yay!
yay!
yay!
yay!
yay!
yay!
yay!
yay!
yay!
yay!
yay!
yay!
yay!
yay!
yay!
yay!
yay!
yay!
yay!
yay!
yay!
yay!
yay!
yay!
yay!
yay!
yay!
yay!
yay!
yay!
yay!
yay!
yay!
yay!
yay!
yay!
yay!
yay!
yay!
yay!
yay!
yay!
yay!
yay!
yay!
yay!
yay!
yay!
yay!
yay!
yay!
yay!
yay!
yay!
yay!
yay!
yay!
yay!
yay!
yay!
yay!
yay!
yay!
yay!
yay!
yay!
yay!
yay!
yay!
yay!
yay!
yay!
yay!
yay!
yay!


In [ ]:
# SAVE THE RESNET50 FEATURE DATA
filename = #TODO
path = # TODO
np.save(filename, path)